In [29]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
import faiss
from llama_index import GPTFaissIndex, SimpleDirectoryReader, LangchainEmbedding

In [32]:
model_name = "hkunlp/instructor-large"
embed_instruction = "Represent the Hugging Face library documentation"
query_instruction = "Query the most relevant piece of information from the Hugging Face documentation"
embedding = HuggingFaceInstructEmbeddings(model_name=model_name, embed_instruction=embed_instruction, query_instruction=query_instruction)
embedding = LangchainEmbedding(embedding)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: hkunlp/instructor-large


load INSTRUCTOR_Transformer


INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


max_seq_length  512


In [33]:
d = 768
faiss_index = faiss.IndexFlatL2(d)
documents = SimpleDirectoryReader('./datasets/hf/').load_data()
index = GPTFaissIndex(documents, embed_model=embedding, faiss_index=faiss_index)
index.query("How to install transformers library?")
index.save_to_disk(
    'index_faiss.json', 
    faiss_index_save_path="index_faiss_core.index"
)
index.query("How to install transformers library?")

INFO:llama_index.token_counter.token_counter:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_documents] Total embedding token usage: 492221 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 3888 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 7 tokens
